In [1]:
from datetime import timedelta
import sys
sys.path.insert(0,'..')
import pandas as pd
import psychrolib
from doe_xstock.database import SQLiteDatabase

In [2]:
DATABASE_FILEPATH = '/Users/kingsleyenweye/Desktop/INTELLIGENT_ENVIRONMENT_LAB/doe_xstock/database.db'
DATABASE = SQLiteDatabase(DATABASE_FILEPATH)

In [4]:
data_filepath = '/Users/kingsleyenweye/Desktop/INTELLIGENT_ENVIRONMENT_LAB/doe_xstock/doe_xstock/data/TX_SPs_Final_0909.parquet'
data = pd.read_parquet(data_filepath)
data = data.rename(columns={
    'Identifier':'building_name',
    'TemperatureExpectedCool':'cooling_setpoint',
    'TemperatureExpectedHeat':'heating_setpoint',
})

# build timestamp
data['timestamp'] = pd.to_datetime(data['date'])
data['timestamp'] = data.apply(lambda x: x['timestamp'] + timedelta(hours=x['hour']), axis=1)
data['timestamp'] = data['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

# convert to celcius
for c in ['cooling_setpoint', 'heating_setpoint']:
    data[c] = (data[c] - 32)*5/9

# location
data['location'] = 'TX, Travis County'

location_query = """
INSERT OR IGNORE INTO ecobee_location (name)
VALUES (:location)
;"""
location_values = data.groupby('location').size().reset_index().to_dict('records')

building_query = """
INSERT INTO ecobee_building (location_id, name)
VALUES ((SELECT id FROM ecobee_location WHERE name = :location), :building_name)
ON CONFLICT (name) DO UPDATE SET location_id = EXCLUDED.location_id
;"""
building_values = data.groupby(['building_name','location']).size().reset_index().to_dict('records')

timeseries_query = """
INSERT INTO ecobee_timeseries (building_id, timestamp, cooling_setpoint, heating_setpoint)
VALUES ((SELECT id FROM ecobee_building WHERE name = :building_name), :timestamp, :cooling_setpoint, :heating_setpoint)
ON CONFLICT (building_id, timestamp) DO UPDATE SET
    (cooling_setpoint, heating_setpoint) = (EXCLUDED.cooling_setpoint, EXCLUDED.heating_setpoint)
;"""
timeseries_values = data.to_dict('records')

DATABASE.insert_batch(
    [location_query, building_query, timeseries_query],
    [location_values, building_values, timeseries_values]
)